In [1]:
import numpy as np
import random
import time
import pygame
class Grid:
    def __init__(self,width,height):
        self.size=(width,height)
        self.grid=np.zeros(self.size,dtype=np.int8)
        self.visible=np.zeros(self.size,dtype=np.int8)-9
    # Get neighbors of a cell
    def neighbors(self,positions:tuple):
        positions=np.array(positions)
        stack=[]
        for addx in range(-1,2,1):
            for addy in range(-1,2,1):
                if(addx!=0 or addy !=0):
                    stack.append(positions+np.array([addx,addy]))
                
        stack = np.vstack(stack)
        X=stack.T[0]
        stack=stack[np.logical_and(X>=0, X<=self.size[0]-1)]
        Y=stack.T[1]
        stack=stack[np.logical_and(Y>=0, Y<=self.size[1]-1)]
        return stack
    def reset(self):
        self.visible=np.zeros(self.size,dtype=np.int8)-9
    
    def rclick(self,position):
        clicked=self.visible[position]
        if(clicked==-1):
            self.visible[position]=-9
        else:
            self.visible[position]=-1
            
    def lclick(self,position):
        position=tuple(position)
        clicked=self.grid[position]
        self.visible[position]=clicked
        if(clicked==-1):
            print("you loose")
            self.visible=self.grid
            self.visible[position]=-9
            return False
        elif(clicked==0):
            neighbors=self.neighbors([position])
            neighbors=neighbors[self.visible[tuple(neighbors.T)]==-9]
            [self.lclick(n) for n in neighbors]
        return True
    # Put bombs in random locations
    def randomize(self,n_bombs):
        sample= random.sample(range(self.size[0]*self.size[1]),n_bombs)
        grid=self.grid.reshape(-1)
        grid[sample]=-1
        self.grid=grid.reshape(self.size)
    # Generate the grid to play
    def generate(self,n_bombs):
        self.randomize(n_bombs)
        stack=[]
        G=self.grid
        Bombs=np.argwhere(G==-1)
        stack=self.neighbors(Bombs)
        stack=stack[G[tuple(stack.T)]!=-1]
        for s in stack:
            G[tuple(s)]+=1
        self.grid=G

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
class Game:
    def __init__(self,Grid,Width=900,Height=500,border=10,top_border=20):
        self.Grid=Grid
        self.visible=Grid.visible
        pygame.init()
        pygame.display.set_caption("MineSweeper")
        self.Win=pygame.display.set_mode((Width+2*border,Height+top_border+border))
        
        images=['empty','flag','Grid', 'grid1', 'grid2', 'grid3', 'grid4', 'grid5', 'grid6', 'grid7', 'grid8','mine','mineClicked','mineFalse']
        self.sprite={img:pygame.image.load(f"Sprites/{img}.png") for img in images}
    def set_background_color(self,color):
        self.Win.fill(color)
    def draw(self):
        pygame.display.update()
    def wait(self,sec):
        time.sleep(sec)
    def quit(self):
        pygame.quit()

In [12]:
G=Grid(10,10)
G.generate(10)
GG=Game(G)
GG.set_background_color((127,127,127))
GG.Win.blit(GG.sprite['empty'],(0,0))
GG.draw()

GG.wait(4)
GG.quit()